In [33]:
# 闭眼，复制，粘贴，就好
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [34]:
# from fastai import *
from fastai.vision import * # 加载所需的几乎所有工具
from fastai.metrics import error_rate # 补充工具
# from fastai.vision.all import *


In [35]:
# bs = 64 # 一次处理64张图片，但如果内存不够（bus error)，则需要重启kernel, 将64缩小为16，降低内存需求
# bs = 16 # `0` + `0` = 重启kernel
bs = 8

In [36]:
path = untar_data(URLs.PETS); path

Path('C:/Users/jyli39/.fastai/data/oxford-iiit-pet')

In [37]:
print(path)

C:\Users\jyli39\.fastai\data\oxford-iiit-pet


In [38]:
path.ls() # 查看数据文件内容

(#2) [Path('C:/Users/jyli39/.fastai/data/oxford-iiit-pet/annotations'),Path('C:/Users/jyli39/.fastai/data/oxford-iiit-pet/images')]

In [39]:
path_anno = path/'annotations'
path_img = path/'images' # 进一步找到直接存有数据文件夹

In [40]:
fnames = get_image_files(path_img) # 将所有数据图片，做成地址Path,放入一个list中
fnames[:5] # 查看list

(#5) [Path('C:/Users/jyli39/.fastai/data/oxford-iiit-pet/images/Abyssinian_1.jpg'),Path('C:/Users/jyli39/.fastai/data/oxford-iiit-pet/images/Abyssinian_10.jpg'),Path('C:/Users/jyli39/.fastai/data/oxford-iiit-pet/images/Abyssinian_100.jpg'),Path('C:/Users/jyli39/.fastai/data/oxford-iiit-pet/images/Abyssinian_101.jpg'),Path('C:/Users/jyli39/.fastai/data/oxford-iiit-pet/images/Abyssinian_102.jpg')]

In [41]:
np.random.seed(2) # 确保每次训练后，模型都被相同的验证数据验证
pat = r'/([^/]+)_\d+.jpg$' # 用re从文件名中提取图片label标注

In [42]:
# 生成数据集（包含训练集和验证集，测试集可选）
data = ImageDataBunch.from_name_re(path_img, # 图片数据文件夹path
                                   fnames, # 图片path的list
                                   pat, # regexpr 
                                   ds_tfms=get_transforms(), # 图片所需的处理
                                   size=224, # 图片裁剪大小, but having bus error,
#                                    size=56, # 缩小图片
                                   bs=bs # 一次处理图片数量
                                  ).normaliz 

NameError: name 'ImageDataBunch' is not defined

In [ ]:
data.show_batch(rows=3, figsize=(7,6)) # 展示数据

In [ ]:
print(data.classes) # 打印类别名称
len(data.classes),data.c # 类别总数

In [ ]:
#现在我们开始训练模型。模型主干是CNN convolutional neural network 外加一个全链接含一个隐藏层的分类器。之后课程中我们会深入学习CNN，现在只需要知道我们的模型可以“吃进”图片“吐出”种类识别（从37个类别中）。

# 创建一个CNN模型，使用data作为数据，下载和调用resnet34作为模型框架和参数，同时打印错误率
learn = create_cnn(data, models.resnet34, metrics=error_rate) 

In [ ]:
learn.model # 查看模型内在结构

In [ ]:
learn.fit_one_cycle(1) # 训练模型，完整训练数据集一遍

In [ ]:
# learn.save('/kaggle/working/stage-1') # 将模型保存在工作目录下，commit后可下载
learn.save('./fast ai_your pets/stage-1') 

In [ ]:
#我们看看训练成果 首先，看看模型最容易混淆的种类。也就是看看出错是否在情理之中。如果错是犯在了很难分辨的地方，这说明模型还不错。 然后，画出confusion matrix，可以看出分布是不对称的；如果模型在相同的地方反复犯错，而其他类别则没问题。这意味着模型只是针对某些类别易出错。以上都是很正常的现象。
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
doc(interp.plot_top_losses)

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=2)

In [ ]:
#解冻，微调，学习率
# 既然模型工作正常，我们可以开始解冻模型，做更多层的训练
learn.unfreeze()
learn.fit_one_cycle(1)

In [ ]:
# learn.load('/kaggle/working/stage-1');
learn.load('./fast ai_your pets/stage-1');
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, max_lr=slice(1e-6,1e-4))

In [ ]:
# learn.save('/kaggle/working/stage-2')
learn.save('./fast ai_your pets/stage-2')

In [ ]:
#现在我们来训练更大的模型resnet50(50层而非34层）更多细节会在后续课程和这篇论文中了解 resnet paper).
# 基本上，resnet50会表现更好，因为它更深参数更多。采用更大的图片训练，模型可以看到更多特征，有利于模型学习。同时，我们减少一次“喂给”模型的图片数量，避免GPU内存枯竭。
data = ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms=get_transforms(),
                                   size=299, bs=bs//2).normalize(imagenet_stats)
                                   

In [ ]:
learn = create_cnn(data, models.resnet50, metrics=error_rate)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1) # 4 次是正常数量

In [ ]:
# learn.save('/kaggle/working/stage-1-50')
learn.save('./fast ai_your pets/stage-1-50')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, max_lr=slice(1e-6,1e-4))

In [ ]:
#如果效果不好，可以调回之前的模型，继续训练
# learn.load('/kaggle/working/stage-1-50');

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.most_confused(min_val=2)

In [ ]:
#其他制作DataBunch的方法
path = untar_data(URLs.MNIST_SAMPLE); path

In [ ]:
tfms = get_transforms(do_flip=False)
data = ImageDataBunch.from_folder(path, ds_tfms=tfms, size=26)

In [ ]:
data.show_batch(rows=3, figsize=(5,5))

In [ ]:
learn = create_cnn(data, models.resnet18, metrics=accuracy)
learn.fit(1)

In [ ]:
df = pd.read_csv(path/'labels.csv')
df.head()

In [ ]:
data = ImageDataBunch.from_csv(path, ds_tfms=tfms, size=28)

In [ ]:
data.show_batch(rows=3, figsize=(5,5))
data.classes

In [ ]:
data = ImageDataBunch.from_df(path, df, ds_tfms=tfms, size=24)
data.classes

In [ ]:
fn_paths = [path/name for name in df['name']]; fn_paths[:2]

In [ ]:
pat = r"/(\d)/\d+\.png$"
data = ImageDataBunch.from_name_re(path, fn_paths, pat=pat, ds_tfms=tfms, size=24)
data.classes

In [ ]:
data = ImageDataBunch.from_name_func(path, fn_paths, ds_tfms=tfms, size=24,
        label_func = lambda x: '3' if '/3/' in str(x) else '7')
data.classes

In [ ]:
labels = [('3' if '/3/' in str(x) else '7') for x in fn_paths]
labels[:5]

In [ ]:
data = ImageDataBunch.from_lists(path, fn_paths, labels=labels, ds_tfms=tfms, size=24)
data.classes